In [55]:
#!pip install duckdb

In [56]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import os
import duckdb
import kagglehub

### Use the following bash command to find the cached file path
find ~ -name "financial_fraud_detection_dataset.csv"

If you're not able to, then download a copy of the dataset to your machine, unzip it and set the absolute path of the csv file in the next cell.

- Download link:
https://www.kaggle.com/datasets/aryan208/financial-transactions-dataset-for-fraud-detection/data


In [57]:
# Copy and paste the file path of the cached dataset below to read it into a pandas DataFrame
#df = pd.read_csv("/Users/user/.cache/kagglehub/datasets/aryan208/financial-transactions-dataset-for-fraud-detection/versions/1/financial_fraud_detection_dataset.csv")
df = pd.read_csv("/Users/marylopez/.cache/kagglehub/datasets/aryan208/financial-transactions-dataset-for-fraud-detection/versions/1/financial_fraud_detection_dataset.csv")
print(df.head())

  transaction_id                   timestamp sender_account receiver_account  \
0        T100000  2023-08-22T09:22:43.516168      ACC877572        ACC388389   
1        T100001  2023-08-04T01:58:02.606711      ACC895667        ACC944962   
2        T100002  2023-05-12T11:39:33.742963      ACC733052        ACC377370   
3        T100003  2023-10-10T06:04:43.195112      ACC996865        ACC344098   
4        T100004  2023-09-24T08:09:02.700162      ACC584714        ACC497887   

    amount transaction_type merchant_category location device_used  is_fraud  \
0   343.78       withdrawal         utilities    Tokyo      mobile     False   
1   419.65       withdrawal            online  Toronto         atm     False   
2  2773.86          deposit             other   London         pos     False   
3  1666.22          deposit            online   Sydney         pos     False   
4    24.43         transfer         utilities  Toronto      mobile     False   

  fraud_type  time_since_last_transact

## Prepare dataset to work with SQL
The downloaded CSV file containing the original dataset was converted into columnar Parquet files, which are much faster to query. After that, data exploration and cleaning were performed using SQL queries in DuckDB to improve memory efficiency.

### Method:
- Download the dataset to the local machine.
- Connect to the file path in the Jupyter notebook and convert the CSV file into Parquet files (columnar Parquet files are much faster to query).
- Store the Parquet files in a folder within the repository (Parquet files are smaller).
- Run SQL queries directly on the Parquet files without importing them into memory.
- Perform feature engineering using DuckDB SQL or Pandas.
- Save the cleaned and processed Parquet shards/files for use in other notebooks.



In [58]:
# Define file paths
csv_path = "/Users/User/.cache/kagglehub/datasets/aryan208/financial-transactions-dataset-for-fraud-detection/versions/1/financial_fraud_detection_dataset.csv"
csv_path = "/Users/marylopez/.cache/kagglehub/datasets/aryan208/financial-transactions-dataset-for-fraud-detection/versions/1/financial_fraud_detection_dataset.csv"
parquet_path = "./raw_data/financial_fraud_detection_dataset.parquet"
cleaned_parquet_path = "./cleaned_data/cleaned_fraud.parquet"

# 1. Check if source CSV exists
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"CSV file not found at {csv_path}")

print(f"📁 Source CSV: {csv_path}")
print(f"📁 Target Parquet: {parquet_path}")
print(f"📊 Original size: {os.path.getsize(csv_path) / (1024**3):.2f} GB")

📁 Source CSV: /Users/marylopez/.cache/kagglehub/datasets/aryan208/financial-transactions-dataset-for-fraud-detection/versions/1/financial_fraud_detection_dataset.csv
📁 Target Parquet: ./raw_data/financial_fraud_detection_dataset.parquet
📊 Original size: 0.74 GB


In [59]:
# Connect to duckdb and Convert CSV to Parquet with DuckDB

# CREATE THE DIRECTORY parquet_path directory FIRST
os.makedirs(os.path.dirname(parquet_path), exist_ok=True)

# f used to pass in a variable to print or SQL statements
# """ are used for multi-line SQL queries, " is used for single-line SQL queries
con = duckdb.connect()
con.execute(f"""
COPY (SELECT * FROM read_csv_auto('{csv_path}'))
TO '{parquet_path}' (FORMAT 'parquet', COMPRESSION 'zstd');
""")

#  3. Verify the result
if os.path.exists(parquet_path):
    parquet_size = os.path.getsize(parquet_path) / (1024**3)
    compression_ratio = (1 - parquet_size / (os.path.getsize(csv_path) / (1024**3))) * 100
    print(f"📊 Parquet size: {parquet_size:.2f} GB")
    print(f"🎯 Compression ratio: {compression_ratio:.1f}% reduction")
    
    # Quick verification query
    row_count = con.execute(f"SELECT COUNT(*) FROM '{parquet_path}'").fetchone()[0]
    print(f"🔢 Row count in Parquet: {row_count:,}")
else:
    print("❌ Parquet file was not created")

# To Close DB connection, but can be left open for further queries
# con.close()


📊 Parquet size: 0.19 GB
🎯 Compression ratio: 74.6% reduction
🔢 Row count in Parquet: 5,000,000


In [60]:
con = duckdb.connect()

# using DESCRIBE instead of pandas dtypes to avoid loading data into memory
print(con.execute(f"DESCRIBE SELECT * FROM read_parquet('{parquet_path}')").fetch_df())


                    column_name column_type null   key default extra
0                transaction_id     VARCHAR  YES  None    None  None
1                     timestamp   TIMESTAMP  YES  None    None  None
2                sender_account     VARCHAR  YES  None    None  None
3              receiver_account     VARCHAR  YES  None    None  None
4                        amount      DOUBLE  YES  None    None  None
5              transaction_type     VARCHAR  YES  None    None  None
6             merchant_category     VARCHAR  YES  None    None  None
7                      location     VARCHAR  YES  None    None  None
8                   device_used     VARCHAR  YES  None    None  None
9                      is_fraud     BOOLEAN  YES  None    None  None
10                   fraud_type     VARCHAR  YES  None    None  None
11  time_since_last_transaction      DOUBLE  YES  None    None  None
12     spending_deviation_score      DOUBLE  YES  None    None  None
13               velocity_score   

In [61]:
con.execute(f"SELECT * FROM read_parquet('{parquet_path}')LIMIT 5").fetch_df()

,transaction_id,timestamp,sender_account,receiver_account,amount,transaction_type,merchant_category,location,device_used,is_fraud,fraud_type,time_since_last_transaction,spending_deviation_score,velocity_score,geo_anomaly_score,payment_channel,ip_address,device_hash
0,T100000,2023-08-22 09:22:43.516168,ACC877572,ACC388389,343.78,withdrawal,utilities,Tokyo,mobile,False,None,NaN,-0.21,3,0.22,card,13.101.214.112,D8536477
1,T100001,2023-08-04 01:58:02.606711,ACC895667,ACC944962,419.65,withdrawal,online,Toronto,atm,False,None,NaN,-0.14,7,0.96,ACH,172.52.47.194,D2622631
2,T100002,2023-05-12 11:39:33.742963,ACC733052,ACC377370,2773.86,deposit,other,London,pos,False,None,NaN,-1.78,20,0.89,card,185.98.35.23,D4823498
3,T100003,2023-10-10 06:04:43.195112,ACC996865,ACC344098,1666.22,deposit,online,Sydney,pos,False,None,NaN,-0.60,6,0.37,wire_transfer,107.136.36.87,D9961380
4,T100004,2023-09-24 08:09:02.700162,ACC584714,ACC497887,24.43,transfer,utilities,Toronto,mobile,False,None,NaN,0.79,13,0.27,ACH,108.161.108.255,D7637601


## Data Exploration
**Check List:**
- Row/column completeness impact — Compute how many rows remain after dropping rows with any NA and after dropping only rows missing critical fields (e.g., is_fraud, amount) so you can plan sample sizes for training

- Class imbalance and sampling — Measure fraud:non‑fraud ratio and per‑group rates (by merchant, device, country). This informs evaluation metrics and resampling strategies (class weights, SMOTE, stratified sampling)

- Duplicates and identity checks — Look for duplicate transaction_id or repeated (sender, receiver, timestamp, amount) tuples. Duplicates can bias counts and model training

- Outliers and distributions — Inspect amount, time_since_last_transaction, and anomaly scores for extreme values and skew. Decide winsorizing, log transforms, or robust scaling. Visualize with histograms or quantile summaries.

- Timestamp and temporal integrity — Check for timezone issues, future dates, or inconsistent formats. Verify monotonicity for per‑account sequences if we’ll use time‑based features.

- For merchant_category, location, device_used, check unique counts and frequency tails. Rare categories may need grouping into “other” or target encoding.

- Compute correlation matrix for numeric features and check for highly correlated predictors that may harm some models.

- Validate is_fraud and fraud_type consistency; ensure no features leak the label (e.g., fraud_flag derived from is_fraud). Check that features available at prediction time won’t include future info.

- PII and privacy — Identify columns with PII (account IDs, IPs, device hashes). Decide hashing/anonymization and access controls before sharing data.



In [62]:
# Create a Temporal Table for data exploration, to avoid any accidental changes to the original dataset:
con.execute(
    f"""
    CREATE TABLE raw_data AS
    SELECT * 
    FROM read_parquet('{parquet_path}') 
    """
    )

# to see the temporal table raw_data
con.execute(f"""
            SELECT *
            FROM raw_data LIMIT 5
            """).fetch_df()

,transaction_id,timestamp,sender_account,receiver_account,amount,transaction_type,merchant_category,location,device_used,is_fraud,fraud_type,time_since_last_transaction,spending_deviation_score,velocity_score,geo_anomaly_score,payment_channel,ip_address,device_hash
0,T100000,2023-08-22 09:22:43.516168,ACC877572,ACC388389,343.78,withdrawal,utilities,Tokyo,mobile,False,None,NaN,-0.21,3,0.22,card,13.101.214.112,D8536477
1,T100001,2023-08-04 01:58:02.606711,ACC895667,ACC944962,419.65,withdrawal,online,Toronto,atm,False,None,NaN,-0.14,7,0.96,ACH,172.52.47.194,D2622631
2,T100002,2023-05-12 11:39:33.742963,ACC733052,ACC377370,2773.86,deposit,other,London,pos,False,None,NaN,-1.78,20,0.89,card,185.98.35.23,D4823498
3,T100003,2023-10-10 06:04:43.195112,ACC996865,ACC344098,1666.22,deposit,online,Sydney,pos,False,None,NaN,-0.60,6,0.37,wire_transfer,107.136.36.87,D9961380
4,T100004,2023-09-24 08:09:02.700162,ACC584714,ACC497887,24.43,transfer,utilities,Toronto,mobile,False,None,NaN,0.79,13,0.27,ACH,108.161.108.255,D7637601


In [87]:
# List all columns and their types, only for datatype and columns name visualization

con.execute(
    f"""
    PRAGMA table_info('raw_data')
    """
).fetch_df()

,cid,name,type,notnull,dflt_value,pk
0,0,transaction_id,VARCHAR,False,None,False
1,1,timestamp,TIMESTAMP,False,None,False
2,2,sender_account,VARCHAR,False,None,False
3,3,receiver_account,VARCHAR,False,None,False
4,4,amount,DOUBLE,False,None,False
5,5,transaction_type,VARCHAR,False,None,False
6,6,merchant_category,VARCHAR,False,None,False
7,7,location,VARCHAR,False,None,False
8,8,device_used,VARCHAR,False,None,False
9,9,is_fraud,BOOLEAN,False,None,False


**Conclusion:**

- VARCHAR: transaction_id, sender_account, receiver_account, transaction_type, merchant_category, location, device_used, fraud_type, payment_channel, ip_address, device_hash
- TIMESTAMP: timestamp
- Numerical (Incluye DOUBLE, BIGINT, INTEGER, FLOAT): amount, time_since_last_transaction, spending_deviation_score, velocity_score, geo_anomaly_score
- BOOLEAN: is_fraud

All columns are stored in a consistent internal format, therefore it does not need to be adjusted.

### 1. Determine data collection period


In [63]:
# check to see if there are any rows with erroneous timestamps set in the future (> 2025)
# check to ensure timestamp consistency format

# sort timestamp column by highest to lowest, include transaction_id column (to indentify unique rows in case of located error)
con.execute(f"""
            SELECT transaction_id, timestamp 
            FROM raw_data
            ORDER BY timestamp DESC
            """).fetch_df()

,transaction_id,timestamp
0,T1280251,2024-01-01 22:58:30.131850
1,T4841484,2024-01-01 22:54:21.281089
2,T2469382,2024-01-01 22:53:53.515483
3,T341139,2024-01-01 22:52:56.620090
4,T681385,2024-01-01 22:50:49.475634
...,...,...
4999995,T3517687,2023-01-01 00:23:15.259766
4999996,T648800,2023-01-01 00:21:19.560899
4999997,T3001064,2023-01-01 00:12:48.028557
4999998,T114745,2023-01-01 00:11:36.452582


## Conclusion:

- The data set spans the period of one year from 2023-01-01 to 2024-01-01.
- There are no future date values in the timestamps.
- The timestamp column is stored in a consistent internal format (as TIMESTAMP type) therefore it does not need to be adjusted.

### 2. Check for distinct values by column:


In [64]:
# Get column names from DuckDB internal metadata
column_info = con.execute("""
    SELECT column_name
    FROM duckdb_columns
    WHERE table_name = 'raw_data'
    ORDER BY column_index
""").fetchall()

# Extract column names into a Python list
columns = [row[0] for row in column_info]

# Dictionary to store results
results = {}

# Loop through each column and count distinct values
for col in columns:
    query = f"""
        SELECT COUNT(DISTINCT "{col}") AS distinct_count
        FROM raw_data
    """
    count = con.execute(query).fetchone()[0]
    results[col] = count

# Convert results into a pandas DataFrame
results_df = pd.DataFrame(
    results.items(),
    columns=["column", "distinct_count"]
)

print(results_df)


                         column  distinct_count
0                transaction_id         5000000
1                     timestamp         4999998
2                sender_account          896513
3              receiver_account          896639
4                        amount          217069
5              transaction_type               4
6             merchant_category               8
7                      location               8
8                   device_used               4
9                      is_fraud               2
10                   fraud_type               1
11  time_since_last_transaction         4103487
12     spending_deviation_score             917
13               velocity_score              20
14            geo_anomaly_score             101
15              payment_channel               4
16                   ip_address         4997068
17                  device_hash         3835723


In [65]:
# Check for logic relating to duplicate/distinct values
# Run a loop to query the distinct values for each column, and return the results in a dictionary.

unique_values = {}

for col in df.columns:
    query = f"""
        SELECT DISTINCT {col}
        FROM raw_data
        ORDER BY {col}
    """
    unique_values[col] = con.execute(query).fetch_df()

unique_values


{'transaction_id':         transaction_id
 0              T100000
 1             T1000000
 2             T1000001
 3             T1000002
 4             T1000003
 ...                ...
 4999995        T999995
 4999996        T999996
 4999997        T999997
 4999998        T999998
 4999999        T999999
 
 [5000000 rows x 1 columns],
 'timestamp':                          timestamp
 0       2023-01-01 00:09:26.241974
 1       2023-01-01 00:11:36.452582
 2       2023-01-01 00:12:48.028557
 3       2023-01-01 00:21:19.560899
 4       2023-01-01 00:23:15.259766
 ...                            ...
 4999993 2024-01-01 22:50:49.475634
 4999994 2024-01-01 22:52:56.620090
 4999995 2024-01-01 22:53:53.515483
 4999996 2024-01-01 22:54:21.281089
 4999997 2024-01-01 22:58:30.131850
 
 [4999998 rows x 1 columns],
 'sender_account':        sender_account
 0           ACC100000
 1           ACC100001
 2           ACC100002
 3           ACC100003
 4           ACC100004
 ...               ...
 896508 

## Analysis of distinct values by column:

**transaction_id:** 5 million unique values, this is logical as there are 5 million rows and each transaction should have its own unique value.

**timestamp:** 4,999,998 unique values, no nulls values so that means there is two transactions that occurred at the same time as other transactions. To be broken down into month, day of week, hour during feature engineering.

**sender_account:** 896,513 unique values (may be hashed for PI reasons)

**receiver_account:** 896639 unique values (may be hashed for PI reaons)

**amount:** 217,068 unique values that range from 0.01 to 3520.57. We may want to consider converting amount into ranges or categories of some sort when feature engineering.

**transaction_type:** 4 unique values; deposit, payment, transfer, withdrawal

**merchant_category:** 8 unique values; entertainment, grocery, online, other, restaurant, retail, travel, utilities

**location:** 8 unique values; Berlin, Dubai, London, New York, Singapore, Sydney, Tokyo, Toronto

**device_used:** 4 unique values;  atm, mobile, pos, web

**is-fraud:** 2 unique values; 0 = false and 1 = true

**fraud_type:** 2 unique values; card_not_present and none. This column offers little value - to be deleted.

**time_since_last transaction:** 4,103,488 unique values. Ranges from -8777.814182 to 8757.758483 We may want to convert into range or categories of some sort when feature engineering (ex. less than one minute, less than 5 minutes etc).

**spending_deviation_score:** 917 unique values; raning from -5.26 to 5.02

**velocity_score:** 20 unique values; ranges from 1-20

**geo_anomaly_score:** 101 unique values; ranges from 0-1 (decimal values)

**payment_channel:** 4 unique values;  ACH, UPI, card, wire_transfer

**ip_address:** 4,997,068 unique values (to be hashed for PI reasons)

**device_hash:** 3,835,723 unique values


#### Check the relevance of repeat values on sender_account, receiver_account, ip_address, device_hash
sender_account (896,513 unique values) and receiver_account (896,639 unique values) come from a dataset of 5 million transactions. It is important to check whether these values repeat within the fraud‑positive cases. The same applies to ip_address (4,997,068 unique values) and device_hash (3,835,723 unique values).

In [ ]:
con.execute("""
    SELECT 
        'sender_account' AS column_name,
        COUNT(*) AS repeated_values
    FROM (
        SELECT sender_account
        FROM raw_data
        WHERE is_fraud = TRUE
        GROUP BY sender_account
        HAVING COUNT(*) > 1
    )
    
    UNION ALL
    
    SELECT 
        'receiver_account' AS column_name,
        COUNT(*) AS repeated_values
    FROM (
        SELECT receiver_account
        FROM raw_data
        WHERE is_fraud = TRUE
        GROUP BY receiver_account
        HAVING COUNT(*) > 1
    )
    
    UNION ALL
    
    SELECT 
        'ip_address' AS column_name,
        COUNT(*) AS repeated_values
    FROM (
        SELECT ip_address
        FROM raw_data
        WHERE is_fraud = TRUE
        GROUP BY ip_address
        HAVING COUNT(*) > 1
    )
    
    UNION ALL
    
    SELECT 
        'device_hash' AS column_name,
        COUNT(*) AS repeated_values
    FROM (
        SELECT device_hash
        FROM raw_data
        WHERE is_fraud = TRUE
        GROUP BY device_hash
        HAVING COUNT(*) > 1
    );
""").fetch_df()


,column_name,repeated_values
0,sender_account,16337
1,receiver_account,15604
2,ip_address,6
3,device_hash,1757


In [98]:
repetition_stats_df = con.execute("""
    WITH fraud_data AS (
        SELECT *
        FROM raw_data
        WHERE is_fraud = TRUE
    ),

    repetition_stats AS (

        -- sender_account
        SELECT 
            'sender_account' AS column_name,
            MAX(cnt) AS max_repetitions
        FROM (
            SELECT sender_account, COUNT(*) AS cnt
            FROM fraud_data
            GROUP BY sender_account
        )

        UNION ALL

        -- receiver_account
        SELECT 
            'receiver_account' AS column_name,
            MAX(cnt) AS max_repetitions
        FROM (
            SELECT receiver_account, COUNT(*) AS cnt
            FROM fraud_data
            GROUP BY receiver_account
        )

        UNION ALL

        -- ip_address
        SELECT 
            'ip_address' AS column_name,
            MAX(cnt) AS max_repetitions
        FROM (
            SELECT ip_address, COUNT(*) AS cnt
            FROM fraud_data
            GROUP BY ip_address
        )

        UNION ALL

        -- device_hash
        SELECT 
            'device_hash' AS column_name,
            MAX(cnt) AS max_repetitions
        FROM (
            SELECT device_hash, COUNT(*) AS cnt
            FROM fraud_data
            GROUP BY device_hash
        )
    )

    SELECT *
    FROM repetition_stats
""").fetch_df()

repetition_stats_df

,column_name,max_repetitions
0,sender_account,7
1,receiver_account,5
2,ip_address,2
3,device_hash,3


#### Check for how many counts have 7 repetitions, how many have 6... etc in Fraud = TRUE

In [100]:
# Columns to analyze
columns = ["sender_account", "receiver_account"]

# Dictionary to store histogram results
results = {}

# Loop through each selected column and compute repetition counts
for col in columns:
    # Count how many distinct values appear N times when is_fraud = TRUE
    query = f"""
        SELECT repetition_count, COUNT(*) AS num_values
        FROM (
            SELECT "{col}", COUNT(*) AS repetition_count
            FROM raw_data
            WHERE is_fraud = TRUE
            GROUP BY "{col}"
        )
        GROUP BY repetition_count
        ORDER BY repetition_count DESC
    """
    
    histogram = con.execute(query).fetchall()
    results[col] = histogram

# Convert results into a pandas DataFrame
rows = []
for col, hist in results.items():
    for repetition_count, num_values in hist:
        rows.append({
            "column": col,
            "repetition_count": repetition_count,
            "num_values": num_values
        })

results_df = pd.DataFrame(rows)

print(results_df)


              column  repetition_count  num_values
0     sender_account                 7           1
1     sender_account                 5           2
2     sender_account                 4          70
3     sender_account                 3        1091
4     sender_account                 2       15173
5     sender_account                 1      145637
6   receiver_account                 5           1
7   receiver_account                 4          46
8   receiver_account                 3         925
9   receiver_account                 2       14632
10  receiver_account                 1      147325


**Goal** 
- Duplicates and identity checks — Look for duplicate transaction_id or repeated (sender, receiver, timestamp, amount) tuples. Duplicates can bias counts and model training

**Conclusion:**

There were repeated values found in several features: sender_account had 16,337 repeated values with a maximum of 7 repetitions; receiver_account had 15,604 repeated values with a maximum of 5 repetitions; ip_address had 6 repeated values with a maximum of 2 repetitions; and device_hash had 1,757 repeated values with a maximum of 3 repetitions.

The results suggest that fraudulent activity in this dataset is highly concentrated in specific accounts, moderately concentrated in certain devices, and minimally traceable through IP addresses, which aligns with typical fraud behaviors involving account compromise, device reuse, and IP obfuscation.

Both sender_account and receiver_account show a highly skewed distribution of repetitions. The vast majority of accounts appear only once or twice, while very few accounts repeat multiple times. In sender_account, only one value appears 7 times and two values appear 5 times, compared with more than 145,000 values that appear only once. A similar pattern is observed in receiver_account, where just one value appears 5 times and most values occur only once. This indicates that repeated accounts are extremely rare, and the dataset is dominated by unique or low‑frequency account identifiers.


### 3. Check for any random spaces that may exist and remove them.

In [88]:
# Select only text columns (VARCHAR)
con.execute(
    f"""
    SELECT name AS column_name
    FROM pragma_table_info('raw_data')
    WHERE type ILIKE '%VARCHAR%'
    """
).fetch_df()

,column_name
0,transaction_id
1,sender_account
2,receiver_account
3,transaction_type
4,merchant_category
5,location
6,device_used
7,fraud_type
8,payment_channel
9,ip_address


In [68]:
# Detect Leading or trailing spaces  WITH CONCAT QUERY
con.execute("""
    SELECT COUNT(*) AS rows_with_spaces
    FROM raw_data
    WHERE CONCAT(
        transaction_id, sender_account, receiver_account,
        transaction_type, merchant_category, location,
        device_used, fraud_type, payment_channel,
        ip_address, device_hash
    ) <> TRIM(CONCAT(
        transaction_id, sender_account, receiver_account,
        transaction_type, merchant_category, location,
        device_used, fraud_type, payment_channel,
        ip_address, device_hash
    ));
""").fetch_df()

,rows_with_spaces
0,0


- **Conclusion:**  There were not found any random space

### 4. Check which columns contain NULL values, and number of NULL values per column.



In [69]:
# Select  name of columns
con.execute(
    f"""
    SELECT name AS column_name
    FROM pragma_table_info('raw_data')
    """
).fetch_df()

,column_name
0,transaction_id
1,timestamp
2,sender_account
3,receiver_account
4,amount
5,transaction_type
6,merchant_category
7,location
8,device_used
9,is_fraud


In [70]:
# Found columns with null values

con.execute(
    f"""
    SELECT
        COUNT(*) FILTER (WHERE transaction_id IS NULL)              AS transaction_id_nulls,
        COUNT(*) FILTER (WHERE timestamp IS NULL)                  AS timestamp_nulls,
        COUNT(*) FILTER (WHERE sender_account IS NULL)               AS sender_account_nulls,
        COUNT(*) FILTER (WHERE receiver_account IS NULL)             AS receiver_account_nulls,
        COUNT(*) FILTER (WHERE amount IS NULL)                       AS amount_nulls,
        COUNT(*) FILTER (WHERE transaction_type IS NULL)             AS transaction_type_nulls,
        COUNT(*) FILTER (WHERE merchant_category IS NULL)            AS merchant_category_nulls,
        COUNT(*) FILTER (WHERE location IS NULL)                     AS location_nulls,
        COUNT(*) FILTER (WHERE device_used IS NULL)                  AS device_used_nulls,
        COUNT(*) FILTER (WHERE is_fraud IS NULL)                     AS is_fraud_nulls,
        COUNT(*) FILTER (WHERE fraud_type IS NULL)                   AS fraud_type_nulls,
        COUNT(*) FILTER (WHERE time_since_last_transaction IS NULL)  AS time_since_last_transaction_nulls,
        COUNT(*) FILTER (WHERE spending_deviation_score IS NULL)     AS spending_deviation_score_nulls,
        COUNT(*) FILTER (WHERE velocity_score IS NULL)               AS velocity_score_nulls,
        COUNT(*) FILTER (WHERE geo_anomaly_score IS NULL)            AS geo_anomaly_score_nulls,
        COUNT(*) FILTER (WHERE payment_channel IS NULL)              AS payment_channel_nulls,
        COUNT(*) FILTER (WHERE ip_address IS NULL)                   AS ip_address_nulls,
        COUNT(*) FILTER (WHERE device_hash IS NULL)                  AS device_hash_nulls
FROM raw_data
"""
).fetch_df()

,transaction_id_nulls,timestamp_nulls,sender_account_nulls,receiver_account_nulls,amount_nulls,transaction_type_nulls,merchant_category_nulls,location_nulls,device_used_nulls,is_fraud_nulls,fraud_type_nulls,time_since_last_transaction_nulls,spending_deviation_score_nulls,velocity_score_nulls,geo_anomaly_score_nulls,payment_channel_nulls,ip_address_nulls,device_hash_nulls
0,0,0,0,0,0,0,0,0,0,0,4820447,896513,0,0,0,0,0,0


#### Check number of positive and negative fraud cases and count Null values if is_fraud=TRUE

In [71]:
# Check all distinct values and their frequencies in is_fraud
con.execute(
    f"""
    SELECT
        is_fraud,
        COUNT(*) AS cnt
    FROM raw_data
    GROUP BY is_fraud
    ORDER BY cnt DESC
    """
).fetch_df()


,is_fraud,cnt
0,False,4820447
1,True,179553


In [ ]:
# Check ratio of fraud:non-fraud cases
# fraud_ratio = fraud cases / total cases
#no_fraud_ratio no graud cases / total
con.execute(f"""
    SELECT
        SUM(CASE WHEN is_fraud = TRUE THEN 1 ELSE 0 END) *1.0 / COUNT(*) AS fraud_ratio,
        SUM(CASE WHEN is_fraud = FALSE THEN 1 ELSE 0 END) *1.0 / COUNT(*) AS no_fraud_ratio
    FROM raw_data""").fetch_df()

,fraud_ratio,no_fraud_ratio
0,0.035911,0.964089


#### per‑group rates (transaction_type, merchant_category, location, device_used, payment_channel)


In [137]:
transaction_types = ["transfer", "withdrawal", "deposit", "payment"]

queries = []

for t in transaction_types:
    q = f"""
        SELECT
            '{t}' AS transaction_type_name,

            -- Rate: out of all fraud cases, what proportion belongs to this transaction type
            CASE
                WHEN SUM(CASE WHEN is_fraud = TRUE THEN 1 ELSE 0 END) = 0
                THEN NULL
                ELSE 
                    SUM(CASE WHEN is_fraud = 1 AND transaction_type = '{t}' THEN 1 ELSE 0 END) * 1.0
                    / SUM(CASE WHEN is_fraud = TRUE THEN 1 ELSE 0 END)
            END AS rate_transaction_type_within_fraud,

            -- Rate: out of all transactions of this type, what proportion is fraud
            CASE
                WHEN SUM(CASE WHEN transaction_type = '{t}' THEN 1 ELSE 0 END) = 0
                THEN NULL
                ELSE
                    SUM(CASE WHEN transaction_type = '{t}' AND is_fraud = 1 THEN 1 ELSE 0 END) * 1.0
                    / SUM(CASE WHEN transaction_type = '{t}' THEN 1 ELSE 0 END)
            END AS fraud_rate_within_transaction_type

        FROM raw_data
    """
    queries.append(q)

final_query = " UNION ALL ".join(queries)

df = con.execute(final_query).fetch_df()
df



,transaction_type_name,rate_transaction_type_within_fraud,fraud_rate_within_transaction_type
0,transfer,0.252449,0.036253
1,withdrawal,0.249921,0.035938
2,deposit,0.249431,0.035812
3,payment,0.248200,0.035640


Transaction-type Conclusions
- 25.24% of all fraudulent transactions are transfers.
- 3.63% of all transfer transactions are fraudulent.
- 24.99% of all fraudulent transactions are withdrawals.
- 3.59% of all withdrawal transactions are fraudulent.
- 24.94% of all fraudulent transactions are deposits.
- 3.58% of all deposit transactions are fraudulent.
- 24.82% of all fraudulent transactions are payments.
- 3.56% of all payment transactions are fraudulent.

In [139]:

merchant_category = ["entertainment", "other", "grocery", "travel", "retail", "restaurant", "online", "utilities"]

queries = []

for t in merchant_category:
    q = f"""
        SELECT
            '{t}' AS merchant_category_name,

            -- Rate: Among all fraudulent transactions, what proportion belongs to this merchant category?
            CASE
                WHEN SUM(CASE WHEN is_fraud = TRUE THEN 1 ELSE 0 END) = 0
                THEN NULL
                ELSE 
                    SUM(CASE WHEN is_fraud = 1 AND merchant_category = '{t}' THEN 1 ELSE 0 END) * 1.0
                    / SUM(CASE WHEN is_fraud = TRUE THEN 1 ELSE 0 END)
            END AS rate_merchant_category_within_fraud,

            -- Rate: Among all transactions in this merchant category, what proportion is fraudulent?
            CASE
                WHEN SUM(CASE WHEN merchant_category = '{t}' THEN 1 ELSE 0 END) = 0
                THEN NULL
                ELSE
                    SUM(CASE WHEN merchant_category = '{t}' AND is_fraud = 1 THEN 1 ELSE 0 END) * 1.0
                    / SUM(CASE WHEN merchant_category = '{t}' THEN 1 ELSE 0 END)
            END AS fraud_rate_within_merchant_category

        FROM raw_data
    """
    queries.append(q)

final_query = " UNION ALL ".join(queries)

df = con.execute(final_query).fetch_df()
df


,merchant_category_name,rate_merchant_category_within_fraud,fraud_rate_within_merchant_category
0,entertainment,0.125718,0.036098
1,other,0.125623,0.036113
2,grocery,0.125400,0.036028
3,travel,0.125328,0.035967
4,retail,0.125049,0.035849
5,restaurant,0.124570,0.035760
6,online,0.124331,0.035800
7,utilities,0.123980,0.035670


In [140]:
location_list = ["Toronto", "London", "Singapore", "New York", "Sydney", "Berlin", "Tokyo", "Dubai"]

queries = []

for loc in location_list:
    q = f"""
        SELECT
            '{loc}' AS location_name,

            -- Rate: Among all fraudulent transactions, what proportion occurred in this location?
            CASE
                WHEN SUM(CASE WHEN is_fraud = TRUE THEN 1 ELSE 0 END) = 0
                THEN NULL
                ELSE 
                    SUM(CASE WHEN is_fraud = 1 AND location = '{loc}' THEN 1 ELSE 0 END) * 1.0
                    / SUM(CASE WHEN is_fraud = TRUE THEN 1 ELSE 0 END)
            END AS rate_location_within_fraud,

            -- Rate: Among all transactions in this location, what proportion is fraudulent?
            CASE
                WHEN SUM(CASE WHEN location = '{loc}' THEN 1 ELSE 0 END) = 0
                THEN NULL
                ELSE
                    SUM(CASE WHEN location = '{loc}' AND is_fraud = 1 THEN 1 ELSE 0 END) * 1.0
                    / SUM(CASE WHEN location = '{loc}' THEN 1 ELSE 0 END)
            END AS fraud_rate_within_location

        FROM raw_data
    """
    queries.append(q)

final_query = " UNION ALL ".join(queries)

df = con.execute(final_query).fetch_df()
df


,location_name,rate_location_within_fraud,fraud_rate_within_location
0,Toronto,0.125317,0.036039
1,London,0.125189,0.036008
2,Singapore,0.125094,0.035920
3,New York,0.125088,0.035916
4,Sydney,0.125077,0.035926
5,Berlin,0.124949,0.035879
6,Tokyo,0.124866,0.035815
7,Dubai,0.124420,0.035783


In [141]:
device_used_list = ["atm", "pos", "web", "mobile"]

queries = []

for d in device_used_list:
    q = f"""
        SELECT
            '{d}' AS device_used_name,

            -- Rate: Among all fraudulent transactions, what proportion was performed using this device type?
            CASE
                WHEN SUM(CASE WHEN is_fraud = TRUE THEN 1 ELSE 0 END) = 0
                THEN NULL
                ELSE 
                    SUM(CASE WHEN is_fraud = 1 AND device_used = '{d}' THEN 1 ELSE 0 END) * 1.0
                    / SUM(CASE WHEN is_fraud = TRUE THEN 1 ELSE 0 END)
            END AS rate_device_used_within_fraud,

            -- Rate: Among all transactions using this device type, what proportion is fraudulent?
            CASE
                WHEN SUM(CASE WHEN device_used = '{d}' THEN 1 ELSE 0 END) = 0
                THEN NULL
                ELSE
                    SUM(CASE WHEN device_used = '{d}' AND is_fraud = 1 THEN 1 ELSE 0 END) * 1.0
                    / SUM(CASE WHEN device_used = '{d}' THEN 1 ELSE 0 END)
            END AS fraud_rate_within_device_used

        FROM raw_data
    """
    queries.append(q)

final_query = " UNION ALL ".join(queries)

df = con.execute(final_query).fetch_df()
df


,device_used_name,rate_device_used_within_fraud,fraud_rate_within_device_used
0,atm,0.251831,0.036184
1,pos,0.249798,0.035906
2,web,0.249547,0.035844
3,mobile,0.248823,0.035709


In [142]:
payment_channel_list = ["wire_transfer", "UPI", "card", "ACH"]

queries = []

for p in payment_channel_list:
    q = f"""
        SELECT
            '{p}' AS payment_channel_name,

            -- Rate: Among all fraudulent transactions, what proportion used this payment channel?
            CASE
                WHEN SUM(CASE WHEN is_fraud = TRUE THEN 1 ELSE 0 END) = 0
                THEN NULL
                ELSE 
                    SUM(CASE WHEN is_fraud = 1 AND payment_channel = '{p}' THEN 1 ELSE 0 END) * 1.0
                    / SUM(CASE WHEN is_fraud = TRUE THEN 1 ELSE 0 END)
            END AS rate_payment_channel_within_fraud,

            -- Rate: Among all transactions using this payment channel, what proportion is fraudulent?
            CASE
                WHEN SUM(CASE WHEN payment_channel = '{p}' THEN 1 ELSE 0 END) = 0
                THEN NULL
                ELSE
                    SUM(CASE WHEN payment_channel = '{p}' AND is_fraud = 1 THEN 1 ELSE 0 END) * 1.0
                    / SUM(CASE WHEN payment_channel = '{p}' THEN 1 ELSE 0 END)
            END AS fraud_rate_within_payment_channel

        FROM raw_data
    """
    queries.append(q)

final_query = " UNION ALL ".join(queries)

df = con.execute(final_query).fetch_df()
df


,payment_channel_name,rate_payment_channel_within_fraud,fraud_rate_within_payment_channel
0,wire_transfer,0.250812,0.035992
1,UPI,0.250043,0.035950
2,card,0.249982,0.035917
3,ACH,0.249163,0.035784


**Goal:**
- Class imbalance and sampling — Measure fraud:non‑fraud ratio and per‑group rates (by merchant, device, country). This informs evaluation metrics and resampling strategies (class weights, SMOTE, stratified sampling)

**Conclusion**
-  The dataset have a big inmbalance. From 5 millon transactions, there are only 179553 fraud cases, and the other 4820447 are (no fraud) normal transactions.  With a fraud ratio = 0.035911 and a no fraud ratio = 0.964089.  

In [89]:
#  Determine number of null cases where is_fraud = TRUE
con.execute(f"""
    SELECT
        COUNT(*) FILTER (WHERE is_fraud = TRUE) AS positive_fraud_cases,
        COUNT(*) FILTER ( WHERE is_fraud = TRUE AND fraud_type IS NULL) AS fraud_type_null_cases,
        COUNT(*) FILTER (WHERE is_fraud = TRUE AND time_since_last_transaction IS NULL) AS time_transaction_null_cases
    FROM raw_data
            """).fetch_df()

,positive_fraud_cases,fraud_type_null_cases,time_transaction_null_cases
0,179553,0,0


- **Goal:** 
Row/column completeness impact — Compute how many rows remain after dropping rows with any NA and after dropping only rows missing critical fields (e.g., is_fraud, amount) so you can plan sample sizes for training

-  **Conclusion:**
Data exploration identified two (2) features with NULL values: time_since_last_transaction (896513) and fraud_type (4820447). Queries were conducted to understand the number of NULL values ​​per feature and the proportion of NULL and non-NULL values ​​per feature to determine the best way to address the NULL values.


- All NULL values are in the "is_fraud = FALSE" category because this group contains the largest number of observations, with a non‑fraud transaction ratio of 0.96 versus a fraud transaction ratio of 0.035. Therefore, dropping the NULL values does not affect the minority class which is also the clase of interest to found patterns.

### 5. Determine if all positive fraud cases are categorized as “card” under “payment_channel” column 

In [ ]:
#  List all distinct payment_channel values
con.execute(
    f"""
    SELECT DISTINCT payment_channel
    FROM raw_data
    """
).fetch_df()

,payment_channel
0,ACH
1,card
2,wire_transfer
3,UPI


In [ ]:
# Count fraud-positive  and negative cases grouped by payment_channel
con.execute(
    f"""
    SELECT
        payment_channel,
        COUNT(*) FILTER (WHERE is_fraud = '1') AS fraud_possitive_count,
        COUNT(*) FILTER (WHERE is_fraud = '0') AS fraud_negative_count
    FROM raw_data
    GROUP BY payment_channel
    """
).fetch_df()

,payment_channel,fraud_possitive_count,fraud_negative_count
0,ACH,44738,1205503
1,UPI,44896,1203951
2,wire_transfer,45034,1206185
3,card,44885,1204808


In [ ]:
#  Find fraud-positive cases where payment_channel is not 'card'  and compare with payment_channel is 'card'
con.execute(f"""
     SELECT 
        COUNT(*) FILTER (WHERE is_fraud = TRUE AND payment_channel <> 'card') AS non_card_fraud_positive_cases,
        COUNT(*) FILTER (WHERE is_fraud = TRUE AND payment_channel = 'card') AS card_fraud_positive_cases,
        COUNT(*) FILTER (WHERE is_fraud = FALSE AND payment_channel <> 'card') AS non_card_fraud_negative_cases,
        COUNT(*) FILTER (WHERE is_fraud = FALSE AND payment_channel = 'card') AS card_fraud_negative_cases
        FROM raw_data;
            """).fetch_df()


,non_card_fraud_positive_cases,card_fraud_positive_cases,non_card_fraud_negative_cases,card_fraud_negative_cases
0,134668,44885,3615639,1204808


**Conclusion:**
Fraud cases were found across all payment_channel categories, which indicates that all categories are significant.

### 6. Explore negative values in Amount and time_since_last_transaction 

In [76]:
# Select min and max amounts and time_since_last_transaction from is_fraud = TRUE
con.execute(f"""
            SELECT 
                MIN(amount) AS min_amount,
                MAX(amount) AS max_amount,
                MIN(time_since_last_transaction	) AS min_time_last_transaction,
                MAX(time_since_last_transaction	) AS max_time_last_transaction
            FROM raw_data
            WHERE is_fraud = TRUE;
            """).fetch_df()

,min_amount,max_amount,min_time_last_transaction,max_time_last_transaction
0,0.01,3128.14,-8748.166439,8744.774704


In [81]:
# count negative and positive values in time_since_last_transaction from is_fraud = TRUE

con.execute(f"""
    SELECT 
        COUNT (*) FILTER (
            WHERE is_fraud = TRUE
            AND time_since_last_transaction <0
        ) AS negative_values,
        COUNT (*) FILTER (
            WHERE is_fraud = TRUE
            AND time_since_last_transaction >0
        ) AS positive_values,
    FROM raw_data
            """).fetch_df()



,negative_values,positive_values
0,89880,89673


In [82]:
# count negative and positive values in time_since_last_transaction from is_fraud = FALSE

con.execute(f"""
    SELECT 
        COUNT (*) FILTER (
            WHERE is_fraud = FALSE
            AND time_since_last_transaction <0
        ) AS negative_values,
        COUNT (*) FILTER (
            WHERE is_fraud = FALSE
            AND time_since_last_transaction >0
        ) AS positive_values,
    FROM raw_data
            """).fetch_df()

,negative_values,positive_values
0,1961451,1962483


#### Trying to understand negative values in time_since_lat_transaction

In [85]:
# query to identify lowest value of time_since_last_transaction by location

con.execute(f"""
    SELECT
        location,
        MIN(time_since_last_transaction) AS min_time_since_last_transaction
    FROM raw_data
    GROUP BY location
""").fetch_df()


,location,min_time_since_last_transaction
0,Berlin,-8729.251003
1,Tokyo,-8759.095447
2,Sydney,-8777.814182
3,London,-8751.874686
4,New York,-8752.234850
5,Dubai,-8726.950512
6,Toronto,-8759.574443
7,Singapore,-8759.511666


In [86]:
# query to identify count of negative time_since_last_transaction values per location

con.execute(f"""
    SELECT
        location,
        COUNT(*) AS negative_time_value_count
    FROM raw_data
    WHERE time_since_last_transaction < 0
    GROUP BY location
""").fetch_df()

,location,negative_time_value_count
0,New York,255876
1,London,256065
2,Sydney,257089
3,Tokyo,256807
4,Berlin,256537
5,Dubai,256196
6,Singapore,256595
7,Toronto,256166


In [93]:
# grouped by sender account, order by timestamp , re-calculated the difference between the actual and the last transaction on the record. 

con.execute(f"""
            SELECT
                transaction_id,
                sender_account,
                timestamp,
                is_fraud,
                time_since_last_transaction,
                timestamp - LAG(timestamp) OVER (
                    PARTITION BY sender_account
                    ORDER BY timestamp ASC
                ) AS time_since_last_transaction_calc
            FROM raw_data
            """
).fetch_df()

,transaction_id,sender_account,timestamp,is_fraud,time_since_last_transaction,time_since_last_transaction_calc
0,T2070664,ACC800807,2023-10-06 12:59:04.163210,False,NaN,147 days 20:23:21.024656
1,T2922359,ACC800860,2023-02-25 01:12:36.002118,False,-699.238797,NaT
2,T3095293,ACC800860,2023-03-05 02:07:12.723165,False,192.910200,8 days 00:54:36.721047
3,T2270102,ACC800860,2023-03-26 04:26:55.670112,False,-2985.049362,21 days 02:19:42.946947
4,T4853039,ACC800860,2023-04-19 17:08:51.766168,False,-2695.753863,24 days 12:41:56.096056
...,...,...,...,...,...,...
4999995,T4739315,ACC690407,2023-11-21 23:16:50.761459,False,2365.405716,98 days 13:24:20.577933
4999996,T4817814,ACC690538,2023-02-14 03:28:04.700837,False,-1475.812547,NaT
4999997,T4714081,ACC690538,2023-04-16 15:16:49.869462,False,-1100.618584,61 days 11:48:45.168625
4999998,T126194,ACC690538,2023-05-29 21:25:04.553279,False,NaN,43 days 06:08:14.683817


- **Conclusion:**

Negative values were found only in the time_since_last_transaction feature, ranging from -8777.814182 to 8757.758483. According to the dataset authors, this variable represents the number of hours since the user’s previous transaction. The values may reference the approximate number of hours in a year (8760 hours). However, the negative values are illogical—for example, they would imply that some transactions occur in the future relative to the previous one.

Additional queries were conducted to understand why these negative values appear. One possible explanation was that time differences might have been calculated using transactions recorded in different time zones. To test this, the analysis examined whether negative values were concentrated in specific geographic locations. No correlation was found between negative time_since_last_transaction values and any particular location.

Furthermore, in another attempt to interpret these negative values, the data was grouped by sender_account, ordered by timestamp, and the time difference between each transaction and the previous one was recalculated. This analysis also revealed no meaningful pattern, likely because the dataset does not include all transaction records for each user. As a result, there is no reliable timeline available to compute this feature accurately.